### 1/ Scrap-booking...

L'objectif de cette première étape est de scraper les hôtels de la liste des 35 destinations.
Le spider a été généré initialement via scrapy en mode framework (voir dossier 'scrapbooking').
Il est reproduit ici en mode 'notebook' et fonctionne tout autant (à l'heure où nous écrivons ces lignes, en espérant que le site de booking n'aura pas trop évolué pour que les xpath des éléments soient les même...).

Après étude, il a été choisi de boucler sur les destinations en tentant de remplir le champs de recherche de la page booking, à chaque fois, et non pas en modifiant l'url, car cette deuxième solution suppose de partir d'une très longue url comme url de base, et d'itérer grâce à un paramètre '&ss=' inclu au milieu de celle-ci. La manoeuvre paraît assez hasardeuse, et peut-être moins stable sur le long terme que la première méthode qui utilise le champs de recherche.

En revanche, l'url est modifiée, après chaque 'atterissage' sur une page de listes d'hôtels pour ne sélectionner que les hôtels à proprement parler, en ajoutant un '&nflt=ht_id%3D204' (un filtre additionnel de booking) à l'url en place. Ici, cela présente moins de risque car on part d'une url de base entière et on travaille uniquement sur un terminaison optionnelle pour booking.

Seules les 25 premiers hôtels de la première page (filtrées en hôtels et apart'hôtels) sont scrappés.
En effet, selon nous, scraper toutes les pages est inutile (et énergivore) sauf si nous souhaitons prendre un panel beaucoup plus large pour appliquer nos propres critères de sélections des hôtels... Mais nos critères, aussi sophistiqués soient-ils, seront-ils les mêmes que les clients ? Aura-t-il possibilité de faire varier ces critères ? Non, car l'objectif est de promouvoir des destinations avant tout, et ensuite de proposer un petit panel d'hôtel pour ces destinations... pour une sorte de petit widget typiquement disposé sur une page d'accueil ou une page ressource du site Kayak.
Ainsi, nous faisons confiance à Booking (et nos équipe de collègues !) pour avoir sélectionner en première page les hôtels qu'il veut recommander en priorité (le filtre d'ordre d'affichage par défaut est "Nos préférés"). 


In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

import logging

import os

import pandas as pd

Importation de la liste des 35 destinations :

In [2]:
list_src = pd.read_csv('src/top35_list_cities.txt').reset_index()

In [3]:
list_cities = list_src['Cities'].to_list()

Génération du spider :

In [4]:
# - SPIDER -
class BookingSpider(scrapy.Spider):
    name = 'scrap-booking'
    allowed_domains = ['www.booking.com']
    start_urls = ['https://www.booking.com/index.fr.html/']

    
    def parse(self, response):
        # fonction parse d'entrée sur le site - on remplit la première destination de la liste
        # (pour nous, dans ce cas, il s'agit d'une fonction qui ne sera utilisée qu'une fois)
        iter=0
        return scrapy.FormRequest.from_response(
                response,
                formdata={'ss': list_cities[0]}, # - on utilise le champs 'ss' pour remplir le champs avec la première destination
                callback=self.change_url_hostels, 
                meta={'iter':iter} # - on définit une un itérable à 0, itérateur sur les destinations de la liste
            )
    

    def change_url_hostels(self, response):
        # fonction appelée dès qu'on atterit dans une nouvelle page de destination
        # objectif : changer l'url pour ne visualiser que des hôtels

        iter=response.request.meta['iter'] # on récupère notre variable méta : l'itérable

        url_dest=response.url # on récupère l'url en place
        yield response.follow(url_dest + '&nflt=ht_id%3D204', # on ajoute à l'url en place la variable qui filtre les hotels
                                callback=self.parse_my_dest, 
                                meta={'iter':iter})

    
    def parse_my_dest(self, response):
        # fonction appelée sur une page de city avec la liste (exclusivement) des hôtels

        iter=response.request.meta['iter'] # on récupère notre itérable dans le champs méta

        list_hostels_urls=response.xpath('//*[@id="search_results_table"]/div[2]/div/div/div/div[3]/div[*]/div[1]/div[2]/div/div[1]/div[1]/div/div[1]/div/h3/a/@href').getall()
        #on scrap la liste des 25 urls d'hotels de la page en une seule fois, grâce à l' * et à getall()

        for i, link in enumerate(list_hostels_urls):
            # pour chaque lien/hotel de la page, on appelle une fonction de scraop dans la page de l'hotel
            yield response.follow(link,
                                callback=self.in_hostel_page,
                                meta={'city':list_cities[iter],'hostel_url':link}) # transport avec meta du nom de la ville et du lien de l'hotel

        iter+=1
        if iter<=len(list_cities)-1:
        # quand les 25 links/hotels sont scrappés, on itère notre variable en vue de la prochaine destination
        # la prochaine destination est rempli dans le Form de la page en cours
            yield scrapy.FormRequest.from_response(
                response,
                formdata={'ss': list_cities[iter]},
                callback=self.change_url_hostels,
                meta={'iter':iter} # transport avec méta de l'itérable
            )

    def in_hostel_page(self,response):
    # fonction appelée quand on atterit sur une page propre à l'hotel

        # redéploiement des variables méta - qui feront parties du yield final
        city=response.request.meta['city']
        hostel_url=response.request.meta['hostel_url']

        # scrapping de toutes les infos recherchées
        hostel_name=response.xpath('//div[@id="hp_hotel_name"]/div/div/h2/text()').get()
        hostel_img=response.xpath('//div[@id="hotel_main_content"]/div/div/div[3]/a/img/@src').get()
        hostel_alt_img=response.xpath('//div[@id="hotel_main_content"]/div/div/div[3]/a/img/@alt').get()
        hostel_review=response.xpath('//div[@data-testid="review-score-right-component"]/div/text()').get()
        type=response.xpath('//span[@class=\'e2f34d59b1\']/text()').get()
        lat_long=response.xpath('//a[@id=\'hotel_surroundings\']/@data-atlas-latlng').get()
        description=response.xpath('//*[@id=\'property_description_content\']/p[2]/text()').get()
        # ici on compte le nombre d'étoiles ou de carrés de l'hôtel, pour un éventuel classement
        nb_squares=len(response.xpath('//*[@data-testid=\'rating-squares\']/span[*]').getall())
        nb_stars=len(response.xpath('//*[@data-testid=\'rating-stars\']/span[*]').getall())
        
        # yield final
        yield {
                'city':city,
                'hostel_name':hostel_name,
                'hostel_url':hostel_url,
                'hostel_img':hostel_img,
                'hostel_alt_img':hostel_alt_img,
                'hostel_review':hostel_review,
                'nb_squares':nb_squares,
                'nb_stars':nb_stars,
                'hostel_type':type,
                'lat_long':lat_long,
                'description':description
            }

Paramètres pour écraser le fichier s'il est déjà présent :

In [6]:
filename = "hostels_booking2.csv"

# overwrite du fichier 'result' (si existant), pour un nouveau
if filename in os.listdir('results/'):
        os.remove('results/' + filename)


# # - SETTINGS -
process = CrawlerProcess(settings = {
    'USER_AGENT': 'Chrome/97.0',
    'LOG_LEVEL': logging.INFO,
    "AUTOTHROTTLE_ENABLED": True,
    "FEEDS": {
        'results/' + filename: {"format": "csv"}, # récupération du fichier
    }
})

2022-10-14 17:46:00 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-10-14 17:46:00 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 3.4.8, Platform Windows-10-10.0.19043-SP0


Lancement du spider et scraping :

In [7]:

process.crawl(BookingSpider)
process.start()

2022-10-14 17:46:03 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True, 'LOG_LEVEL': 20, 'USER_AGENT': 'Chrome/97.0'}
2022-10-14 17:46:03 [scrapy.extensions.telnet] INFO: Telnet Password: 2275df5d98d07d9b
2022-10-14 17:46:03 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.throttle.AutoThrottle']
2022-10-14 17:46:03 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermid